In [2]:
import json
import random
import uuid
import pandas as pd

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from datetime import date

import numpy as np

In [3]:
# Load config for SQL
with open("sql_config.json") as f:
    config = json.load(f)
    
Base = declarative_base()
engine = create_engine(config["mysql_url"])

df = pd.read_sql("SELECT * FROM silo_details", engine).drop("id", axis= 1)

storage_df = pd.read_sql("SELECT * FROM silo_storage", engine).drop(["id", "silo_current_capacity"], axis= 1)

In [4]:
df.head()

,silo_name,silo_id,silo_address,silo_district,silo_city,silo_state,lease_expiry,contact_person_id,silo_type
0,silo_0,silo_0,a,a,a,Delhi,2022-01-01,person_0,silo
1,silo_1,silo_1,a,a,a,U. P.,2022-01-01,person_1,type c
2,silo_2,silo_2,a,a,a,U. P.,2022-01-01,person_2,silo
3,silo_3,silo_3,a,a,a,Rajasthan,2022-01-01,person_3,warehouse
4,silo_4,silo_4,a,a,a,Rajasthan,2022-01-01,person_4,silo


In [5]:
storage_df.head()

,silo_id,grain_type,capacity
0,silo_0,corn,733
1,silo_1,wheat,942
2,silo_2,wheat,345
3,silo_3,rice,955
4,silo_4,corn,27


In [112]:
def calc_grain_amount(trans_df):
    a = trans_df.groupby(["to_silo_id", "type_grain"]).sum()["value_grain"].reset_index().rename(columns= {"to_silo_id": "silo_id", "value_grain": "sum_grain"})
    try:
        b = trans_df.groupby(["from_silo_id", "type_grain"]).sum()["value_grain"].reset_index().rename(columns= {"from_silo_id": "silo_id", "value_grain": "sub_grain"})
    except:
        b = pd.DataFrame(columns= ["silo_id", "type_grain", "sub_grain"])
    f = pd.merge(a, b, how= "outer", on= ["silo_id", "type_grain"]).fillna(0)
    f["value_grain"] = f["sum_grain"] - f["sub_grain"]
    return f.drop(["sum_grain", "sub_grain"], axis= 1)

In [113]:
trans_df = pd.read_sql("SELECT * FROM transactions", engine).drop("id", axis= 1)
trans_df.columns

Index(['shipment_id', 'from_silo_id', 'from_address_state',
       'from_address_district', 'from_address_city', 'from_address',
       'to_silo_id', 'to_address_state', 'to_address_district',
       'to_address_city', 'to_address', 'transaction_date', 'shipment_date',
       'arrival_date', 'contact_person_id', 'type_grain', 'value_grain',
       'date_recorded', 'recorder_person_id'],
      dtype='object')

In [114]:
for i in range(500):
    ship_id = str(uuid.uuid1())[:10]
    t_date = random.choice(pd.date_range("2020-02-15", "2022-02-15", ))
    silo = random.choice(df["silo_id"].unique())
    t_df = storage_df[storage_df["silo_id"] == silo]
    grain = random.choice(t_df["type_grain"].tolist())
    max_val = t_df[t_df["type_grain"] == grain]["capacity"].iloc[0]
    grain_value = random.randint(1, max_val//2)
    
    values = {
        'shipment_id': ship_id,
        'transaction_date': t_date,
        'shipment_date': t_date + pd.DateOffset(n= 5),
        'arrival_date': t_date + pd.DateOffset(n= 20),
        'contact_person_id': "person_{}".format(i%10),
        'date_recorded': t_date - pd.DateOffset(n= 2),
        'recorder_person_id': "person_{}".format((i*3)%10),
        'type_grain': grain,
        'to_silo_id': silo,
        'value_grain': grain_value,
        'from_address': random.choice(["a", "b", "c", "d"]),
        'from_address_city': random.choice(["a", "b", "c", "d"]),
        'from_address_district': random.choice(["a", "b", "c", "d"]),
        'from_address_state': random.choice(["U. P.", "Delhi", "Gujarat", "Rajasthan"])
    }
    
    trans_df = trans_df.append(values, ignore_index= True)
    
trans_df.head()

,shipment_id,from_silo_id,from_address_state,from_address_district,from_address_city,from_address,to_silo_id,to_address_state,to_address_district,to_address_city,to_address,transaction_date,shipment_date,arrival_date,contact_person_id,type_grain,value_grain,date_recorded,recorder_person_id
0,a521f2b8-4,NaN,Rajasthan,d,a,d,silo_8,NaN,NaN,NaN,NaN,2021-04-29,2021-05-04,2021-05-19,person_0,corn,169,2021-04-27,person_0
1,a53546e6-4,NaN,Gujarat,c,c,d,silo_88,NaN,NaN,NaN,NaN,2021-03-22,2021-03-27,2021-04-11,person_1,corn,12,2021-03-20,person_3
2,a5367f5c-4,NaN,Rajasthan,b,c,c,silo_52,NaN,NaN,NaN,NaN,2020-09-04,2020-09-09,2020-09-24,person_2,corn,333,2020-09-02,person_6
3,a5379002-4,NaN,Rajasthan,a,b,d,silo_54,NaN,NaN,NaN,NaN,2020-05-24,2020-05-29,2020-06-13,person_3,wheat,72,2020-05-22,person_9
4,a538a0d8-4,NaN,Delhi,b,c,b,silo_48,NaN,NaN,NaN,NaN,2021-04-04,2021-04-09,2021-04-24,person_4,corn,15,2021-04-02,person_2


In [115]:
for i in range(500):
    try:
        ship_id = str(uuid.uuid1())[:10]
        t_date = random.choice(pd.date_range("2020-02-15", "2022-02-15", ))
        s_date = t_date + pd.DateOffset(n= 5)
        silo = random.choice(df["silo_id"].unique())
        t_df = trans_df[(trans_df["to_silo_id"] == silo) & (trans_df["arrival_date"] < s_date)]
        t_df = calc_grain_amount(t_df)
        t_df = t_df[t_df["value_grain"] > 0]
        grain = random.choice(t_df["type_grain"].tolist())
        max_val = t_df[t_df["type_grain"] == grain]["value_grain"].iloc[0]
        grain_value = random.randint(1, max_val//4)

        values = {
            'shipment_id': ship_id,
            'transaction_date': t_date,
            'shipment_date': s_date,
            'arrival_date': t_date + pd.DateOffset(n= 20),
            'contact_person_id': "person_{}".format(i%10),
            'date_recorded': t_date - pd.DateOffset(n= 2),
            'recorder_person_id': "person_{}".format((i*3)%10),
            'type_grain': grain,
            'from_silo_id': silo,
            'value_grain': grain_value,
            'to_address': random.choice(["a", "b", "c", "d"]),
            'to_address_city': random.choice(["a", "b", "c", "d"]),
            'to_address_district': random.choice(["a", "b", "c", "d"]),
            'to_address_state': random.choice(["U. P.", "Delhi", "Gujarat", "Rajasthan"])
        }

        trans_df = trans_df.append(values, ignore_index= True)
    except:
        continue
    
trans_df.head()

,shipment_id,from_silo_id,from_address_state,from_address_district,from_address_city,from_address,to_silo_id,to_address_state,to_address_district,to_address_city,to_address,transaction_date,shipment_date,arrival_date,contact_person_id,type_grain,value_grain,date_recorded,recorder_person_id
0,a521f2b8-4,NaN,Rajasthan,d,a,d,silo_8,NaN,NaN,NaN,NaN,2021-04-29,2021-05-04,2021-05-19,person_0,corn,169,2021-04-27,person_0
1,a53546e6-4,NaN,Gujarat,c,c,d,silo_88,NaN,NaN,NaN,NaN,2021-03-22,2021-03-27,2021-04-11,person_1,corn,12,2021-03-20,person_3
2,a5367f5c-4,NaN,Rajasthan,b,c,c,silo_52,NaN,NaN,NaN,NaN,2020-09-04,2020-09-09,2020-09-24,person_2,corn,333,2020-09-02,person_6
3,a5379002-4,NaN,Rajasthan,a,b,d,silo_54,NaN,NaN,NaN,NaN,2020-05-24,2020-05-29,2020-06-13,person_3,wheat,72,2020-05-22,person_9
4,a538a0d8-4,NaN,Delhi,b,c,b,silo_48,NaN,NaN,NaN,NaN,2021-04-04,2021-04-09,2021-04-24,person_4,corn,15,2021-04-02,person_2


In [116]:
a = calc_grain_amount(trans_df[trans_df["arrival_date"] < pd.to_datetime("2023-12-12")])
a[a["value_grain"] < 0]

,silo_id,type_grain,value_grain


In [117]:
trans_df.to_sql("transactions", engine, if_exists= "append", index = False)

In [104]:
a.head()

,silo_id,type_grain,value_grain
0,silo_0,corn,375.0
1,silo_0,maize,280.0
2,silo_0,wheat,592.0
3,silo_1,corn,649.0
4,silo_1,maize,378.0


In [103]:
storage_df.head()

,silo_id,grain_type,capacity
0,silo_0,corn,733
1,silo_1,wheat,942
2,silo_2,wheat,345
3,silo_3,rice,955
4,silo_4,corn,27
